In [1]:
import pandas as pd
import numpy as np

%matplotlib inline
%config Completer.use_jedi = False

In [3]:
ds = pd.read_feather('../data/preprocessed_train.ftr')

In [6]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13150956 entries, 0 to 13150955
Data columns (total 5 columns):
date_block_num    category
shop_id           category
item_id           category
item_cnt_month    float64
item_price        float64
dtypes: category(3), float64(2)
memory usage: 251.6 MB


In [147]:
from keras.models import Sequential
from keras.models import Model as KerasModel
from keras.layers import Input, Dense, Activation, Reshape
from keras.layers import Concatenate
from keras.layers.embeddings import Embedding
from keras.callbacks import ModelCheckpoint
from keras import optimizers


In [20]:
embedding_columns=ds.select_dtypes(include='category').columns
embedding_columns

Index(['date_block_num', 'shop_id', 'item_id'], dtype='object')

In [57]:
n_levels = ds[embedding_columns].nunique()
n_levels

date_block_num       34
shop_id              60
item_id           21807
dtype: int64

In [88]:
ds[embedding_columns].max()

date_block_num       33.0
shop_id              59.0
item_id           22169.0
dtype: float64

In [89]:
n_levels.item_id=22170

In [96]:
n_dimensions=n_levels.copy()
n_dimensions.iloc[range(3)]=[3,3,10]
n_dimensions

date_block_num     3
shop_id            3
item_id           10
dtype: int64

In [97]:
input_block = Input(shape=(1,))
output_block = Embedding(n_levels.date_block_num, n_dimensions.date_block_num, name='block_embedding')(input_block)
output_block = Reshape(target_shape=(n_dimensions.date_block_num,))(output_block)


In [98]:
input_shop = Input(shape=(1,))
output_shop = Embedding(n_levels.shop_id, n_dimensions.shop_id, name='shop_embedding')(input_shop)
output_shop = Reshape(target_shape=(n_dimensions.shop_id,))(output_shop)


In [99]:
input_item = Input(shape=(1,))
output_item = Embedding(n_levels.item_id, n_dimensions.item_id, name='item_embedding')(input_item)
output_item = Reshape(target_shape=(n_dimensions.item_id,))(output_item)


In [100]:
input_model = [input_block, input_shop ,input_item ]

output_embeddings = [output_block,output_shop,output_item]


In [139]:
output_model = Concatenate()(output_embeddings)
output_model = Dense(1000, kernel_initializer="uniform")(output_model)
output_model = Activation('relu')(output_model)
output_model = Dense(500, kernel_initializer="uniform")(output_model)
output_model = Activation('relu')(output_model)
output_model = Dense(1)(output_model)
# output_model = Activation('sigmoid')(output_model)

model = KerasModel(inputs=input_model, outputs=output_model)

In [153]:
sgd = optimizers.SGD(lr=0.05, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_absolute_error', optimizer=sgd)

In [141]:
dsmall=ds.sample(n=10**5)


In [154]:
model.fit(list(dsmall[embedding_columns].values.T),dsmall['item_cnt_month'],validation_split=0.3,epochs=5,verbose=1,batch_size=2048)

Train on 70000 samples, validate on 30000 samples
Epoch 1/5
70000/70000 [==============================] - 10s 148us/step - loss: 0.3189 - val_loss: 0.3221
Epoch 2/5
70000/70000 [==============================] - 11s 150us/step - loss: 0.3128 - val_loss: 0.2914
Epoch 3/5
70000/70000 [==============================] - 11s 162us/step - loss: 0.3131 - val_loss: 0.3090
Epoch 4/5
70000/70000 [==============================] - 12s 165us/step - loss: 0.3118 - val_loss: 0.2743
Epoch 5/5
70000/70000 [==============================] - 11s 162us/step - loss: 0.3117 - val_loss: 0.2769
